<a href="https://colab.research.google.com/github/sogangori/choongang20/blob/master/cnn_2_number.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2 자리수 숫자 CNN 분류기 

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits

In [94]:
x, y = load_digits(return_X_y=True)
x.shape, y.shape, set(y)

((1797, 64), (1797,), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

공유 : http://bitly.kr/1CC8BWNm

학습데이터와 테스트 데이터를 나눕니다

In [95]:
m = len(y)//2
x_train = x[:m]
y_train = y[:m]
x_test = x[m:m*2]
y_test = y[m:m*2]
x_train = np.reshape(x_train, [-1, 8, 8])
x_test = np.reshape(x_test, [-1, 8, 8])
x_train.shape

(898, 8, 8)

이미지를 2개씩 좌우로 붙여서 합성데이터를 만듭니다


In [96]:
x_train_l, x_train_r = np.split(x_train, 2, axis=0) # 데이터를 반 나눕니다
x_train_lr = np.concatenate((x_train_l, x_train_r), axis=2) # 두 데이터를 좌우로 붙입니다
x_test_l, x_test_r = np.split(x_test, 2, axis=0) 
x_test_lr = np.concatenate((x_test_l, x_test_r), axis=2)
y_train_l, y_train_r = np.split(y_train, 2, axis=0) 
y_train = np.stack((y_train_l, y_train_r), -1)
y_test_l, y_test_r = np.split(y_test, 2, axis=0) 
y_test = np.stack((y_test_l, y_test_r), -1)
x_train_lr.shape, x_test_lr.shape, y_train.shape, y_test.shape

((449, 8, 16), (449, 8, 16), (449, 2), (449, 2))

In [0]:
class MyModel(keras.Model): # github.com/sogangori/choongang20/ 
  def __init__(self):    
    super(MyModel, self).__init__()
    self.k = 10 # 클래스 갯수 
    self.seq = 2 # 자릿수
    self.opt = tf.optimizers.SGD(learning_rate=0.01)#Stochatic Gradient Descent 확률적 경사 하강
    self.conv0 = keras.layers.Conv2D(16, [3,3], padding='same', activation=keras.activations.relu)
    self.conv1 = keras.layers.Conv2D(32, [3,3], padding='same', activation=keras.activations.relu)
    self.pool0 = keras.layers.MaxPool2D([2,2], padding='same')
    self.pool1 = keras.layers.MaxPool2D([2,2], padding='same')
    self.flatten = keras.layers.Flatten()
    self.dense = keras.layers.Dense(units=self.k * self.seq)
  
  def call(self, x):
    #x (1797, 64)
    x_4d = tf.reshape(x, [-1,8,8*2,1]) 
    x_4d = tf.cast(x_4d, tf.float32)
    net = self.conv0(x_4d)
    net = self.pool0(net)
    net = self.conv1(net)
    net = self.pool1(net)
    net = self.flatten(net)    
    h = self.dense(net)
    h = tf.reshape(h, [-1, self.seq, self.k]) # 2:두자리수, 10:10개의 클래스 
    h = tf.nn.softmax(h, axis=2)
    return h

  def get_loss(self, y, h):
    #학습할때 nan이 발생하는 경우 값을 clip(자르다) (최소값, 최대값) 
    h = tf.clip_by_value(h, 1e-8, 1 - 1e-8) # h 가 0이나 1이 되지 않도록 하는 안전장치 
    cross_entropy = - (y * tf.math.log(h) + (1 - y) * tf.math.log(1 - h)) 
    loss = tf.reduce_mean(cross_entropy)
    return loss

  def get_accuracy(self, y, h):    
    predict = tf.argmax(h, -1)
    is_equal = tf.equal(y, predict)
    self.acc = tf.reduce_mean(tf.cast(is_equal, tf.float32)) # True > 1, False > 0 로 cast
    self.acc_all = tf.reduce_mean(tf.cast(tf.reduce_all(is_equal, axis=1), tf.float32))

  def fit(self, x, y, epoch=1):
    # x : (m, 8, 16), y: (m, 2)    
    y_hot = tf.one_hot(y, depth=self.k, axis=-1)#(m, 2, 10)  
    for i in range(epoch):
      with tf.GradientTape() as tape: #경사 기록 장치
        h = self.call(x)
        loss = self.get_loss(y_hot, h)        
      grads = tape.gradient(loss, self.trainable_variables) #경사 계산
      self.opt.apply_gradients(zip(grads, self.trainable_variables)) # 가중치에서 경사를 빼기
      self.get_accuracy(y, h)
      if i%10==0:
        print('%d/%d loss:%.3f acc:%.3f acc_all:%.3f'%(i, epoch, loss, self.acc, self.acc_all))
model = MyModel()

In [78]:
model.fit(x_train_lr, y_train, epoch=5000) #학습 

0/5000 loss:0.296 acc:0.290 acc_all:0.100
10/5000 loss:0.296 acc:0.291 acc_all:0.102
20/5000 loss:0.295 acc:0.291 acc_all:0.102
30/5000 loss:0.295 acc:0.292 acc_all:0.102
40/5000 loss:0.295 acc:0.294 acc_all:0.105
50/5000 loss:0.295 acc:0.294 acc_all:0.105
60/5000 loss:0.295 acc:0.295 acc_all:0.105
70/5000 loss:0.295 acc:0.296 acc_all:0.105
80/5000 loss:0.294 acc:0.296 acc_all:0.105
90/5000 loss:0.294 acc:0.297 acc_all:0.107
100/5000 loss:0.294 acc:0.297 acc_all:0.107
110/5000 loss:0.294 acc:0.298 acc_all:0.107
120/5000 loss:0.294 acc:0.298 acc_all:0.107
130/5000 loss:0.294 acc:0.298 acc_all:0.107
140/5000 loss:0.294 acc:0.301 acc_all:0.107
150/5000 loss:0.293 acc:0.300 acc_all:0.107
160/5000 loss:0.293 acc:0.302 acc_all:0.111
170/5000 loss:0.293 acc:0.303 acc_all:0.111
180/5000 loss:0.293 acc:0.303 acc_all:0.111
190/5000 loss:0.293 acc:0.303 acc_all:0.111
200/5000 loss:0.293 acc:0.303 acc_all:0.111
210/5000 loss:0.292 acc:0.305 acc_all:0.111
220/5000 loss:0.292 acc:0.306 acc_all:0.111

In [77]:
# 테스트셋의 성능
h = model(x_test)
model.get_accuracy(y_test, h)
print('개별정확도',model.acc.numpy(),'두자리 모두 맞춘 정확도', model.acc_all.numpy())

개별정확도 0.08017817 두자리 모두 맞춘 정확도 0.0066815144
